# Adversarial Dropout Code
#### This file contains the code used to replicate the results of Adversarial Dropout

In [0]:
import json
import re
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.datasets import mnist
from matplotlib import pyplot as plt
from google.colab import files

## Layers

In [0]:
SEED = 123456
rng = np.random.RandomState(SEED)


def __createWeights(shape, seed=None, name='weight'):
    w_init = tf.contrib.layers.variance_scaling_initializer(seed=seed)
    return tf.get_variable(name + '_w', shape=shape, initializer=w_init)
    

def __createBiases(size, name='bias'):
    return tf.get_variable(name + '_b', shape=[size], initializer=tf.constant_initializer(0.0))

def ReLU(x):
    x = tf.nn.relu(x)
    return x


def LeakyReLU(x, alpha=0.1):
    x = tf.nn.leaky_relu(x, alpha=alpha)
    return x


def MaxPooling(x, ksize=2, stride_length=2, padding='SAME', data_format='NHWC'):
    x = tf.nn.max_pool(x, (1, ksize, ksize, 1), (1, stride_length, stride_length, 1), padding, data_format)
    return x


def GlobalAveragePooling(x):
    x = tf.reduce_mean(x, [1, 2])
    return x


def Dense(x, input_dim, output_dim, seed=None, name='dense'):
    W = __createWeights([input_dim, output_dim], seed, name) 
    b = __createBiases(output_dim, name) 
    x = tf.nn.xw_plus_b(x, W, b)
    return x


def Conv2D(x, filter_size, n_channels, n_filters, stride_length=1, padding='SAME', data_format='NHWC', name='conv'):
    shape = [filter_size, filter_size, n_channels, n_filters]
    W = __createWeights(shape, name=name)
    b = __createBiases(n_filters, name=name)
    x = tf.nn.conv2d(x, filter=W, strides=(1, stride_length, stride_length, 1), padding=padding, data_format=data_format)
    x += b
    return x


def Dropout(x, rate=0.5, is_train=True):
    x = tf.layers.dropout(x, rate=rate, seed=rng.randint(SEED), training=is_train)
    return x


def GaussianNoise(x, sigma=0.15):
    noise = tf.random_normal(shape=tf.shape(x), stddev=sigma)
    x += noise
    return x


def SoftMax(x):
    x = tf.nn.softmax(x)
    return x



'''
Loss functions. Arg 1: Approximation, Arg 2: Labels
'''
def CrossEntropyWithLogits(logits, labels):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss


# Formula: sum(p_i * log(p_i) - p_i * log(q_i))
def KLDivergenceWithLogits(q, p):
    p_soft = SoftMax(p)
    # plogp = tf.reduce_mean(tf.reduce_sum(p_soft * tf.nn.log_softmax(p), 1))
    # plogq = tf.reduce_mean(tf.reduce_sum(p_soft * tf.nn.log_softmax(q), 1))
    distance = tf.reduce_sum(p_soft * tf.nn.log_softmax(p) - p_soft * tf.nn.log_softmax(q))
    # distance = plogp - plogq
    return distance
  
def Batch_Norm(x, axis=-1, is_train=True, momentum=0.999, name='bn'):
    x = tf.layers.batch_normalization(x, axis=axis, training=is_train, momentum=momentum, name=name)
    return x
    

## Building blocks for creating the networks

In [0]:
'''
The model before AdD is applied
'''
def upperBlock(x, conv_size=[32, 64, 128], n_channels=3, is_train=True):
    x = Conv2D(x, filter_size=3, n_channels=n_channels, n_filters=conv_size[0], padding='SAME', name='1a')
    x = Batch_Norm(x, axis=-1, is_train=is_train, momentum=0.999, name='1bna')
    x = ReLU(x)
    x = MaxPooling(x, ksize=2, stride_length=2)
    x = Dropout(x, 0.5, is_train)
    
    x = Conv2D(x, filter_size=3, n_channels=conv_size[0], n_filters=conv_size[1], name='2a')
    x = Batch_Norm(x, axis=-1, is_train=is_train, momentum=0.999, name='2bna')
    x = ReLU(x)
    x = MaxPooling(x, ksize=2, stride_length=2)
    x = Dropout(x, 0.5, is_train)

    x = Conv2D(x, filter_size=3, n_channels=conv_size[1], n_filters=conv_size[2], padding='SAME', name='3a')
    x = Batch_Norm(x, axis=-1, is_train=is_train, momentum=0.999, name='3bna')
    x = ReLU(x)
    x = MaxPooling(x, ksize=2, stride_length=2)
    
    x = tf.reshape(x, [-1, 2048], name='reshaped')
    
    return x


'''
The model after AdD is applied
'''
def lowerBlock(x, n_in=2048, n_out=10, name='fc'):
    x = Dense(x, n_in, 625, name=name + '_1')
    x = Dense(x, 625, n_out, name=name + '_2')
    return x;


'''
Apply adv dropout
'''
def advDropout(x, Jacobian, rate=0.5, sigma=0.05, dim=2048, is_train=True):
    # y: output 
    # mask: current sampled dropout mask 
    # sigma: hyper-parameter for boundary 
    # Jabocian: Jacobian vector (gradient of divergence (or loss function))
    # dim: layer dimension 
    
    Jacobian = tf.reshape(Jacobian, [-1, dim])
    
    # create current dropout mask based on keep_prob
    mask = tf.ones_like(x)
    mask = Dropout(mask, rate, is_train)

    # mask = 0 --> -1 
    mask = 2 * mask - tf.ones_like(mask)

    adv_mask = mask 

    # extract the voxels for which the update conditions hold 
    # mask = 0 and J > 0 
    # or
    # mask = 1 and J < 1 
    abs_jac = tf.abs(Jacobian)
    temp = tf.cast(tf.greater(abs_jac, 0), tf.float32)
    temp = 2 * temp - 1 
    # interested in the cases when temp * mask = -1
    ext = tf.cast(tf.less(mask, temp), tf.float32)

    # keep the voxels that you want to update 
    candidates = abs_jac * ext 
    thres = tf.nn.top_k(candidates, int(dim * sigma * sigma)  + 1)[0][:,-1]

    targets = tf.cast(tf.greater(candidates, tf.expand_dims(thres, -1)), tf.float32)

    # get new mask 
    adv_mask = (mask - targets * 2 * mask + tf.ones_like(mask)) / 2.0

    output = adv_mask * x

    return output, adv_mask

## Models

In [0]:
''' Preprocesses the data '''
def preprocess(data):
    # Mean normalization
    data = (data - 127.5) / 255.
    # Find principal component
    shape = data.shape
    data = data.transpose(0, 2, 3, 1)
    flatx = np.reshape(data, (data.shape[0], data.shape[1] * data.shape[2] * data.shape[3]))
    sigma = np.dot(flatx.T, flatx) / flatx.shape[1]
    U, S, V = np.linalg.svd(sigma)
    pc = np.dot(np.dot(U, np.diag(1. / np.sqrt(S + 0.0001))), U.T)
    # Apply ZCA whitening
    whitex = np.dot(flatx, pc)
    data = np.reshape(whitex, (shape[0], shape[1], shape[2], shape[3]))
    return data


'''
Returns a model without adversarial dropout
'''
def modelWithRandD(x, n_channels=3, is_train=True):
    x = upperBlock(x, n_channels=n_channels, is_train=is_train)
    x = lowerBlock(x, n_in=2048, n_out=10)
    return x


'''
Returns a model with adversarial dropout
'''
def modelWithAdD(x, y, fn_loss=KLDivergenceWithLogits, n_channels=3, is_train=True):
    x = upperBlock(x, n_channels=n_channels, is_train=is_train)
    y_no_adD = lowerBlock(x)
    loss_no_adD = fn_loss(y_no_adD, y)

    # Derivative of loss fn wrt x
    DLoss = tf.gradients(loss_no_adD, [x])
    DLoss = tf.squeeze(tf.stop_gradient(DLoss)) # Stops backpropagation

    Jacobian_approx = DLoss * x
    mask = tf.ones_like(x)

    x, _ = advDropout(x, Jacobian_approx, rate=0.5, is_train=is_train)
    x = lowerBlock(x)

    return x



def CreateBaseModel(x, y, learning_rate=0.001, optimizer=tf.train.AdamOptimizer, n_channels=3, is_train=True):
    logit_rand = modelWithRandD(x, n_channels=n_channels)
    loss = CrossEntropyWithLogits(logit_rand, y)

    opt = optimizer(learning_rate=learning_rate)
    gradients = opt.compute_gradients(loss, tf.trainable_variables())
    train_op = opt.apply_gradients(gradients)

    return train_op, loss, logit_rand


'''
Create the AdD model for training
'''
def CreateAdDModel(x, y, learning_rate=0.001, optimizer=tf.train.AdamOptimizer, lmb=0.01, n_channels=3, is_train=True):
    logit_rand = modelWithRandD(x, n_channels=n_channels, is_train=is_train)
    logit_rand_loss = CrossEntropyWithLogits(logit_rand, y)

    with tf.variable_scope(tf.get_variable_scope(), reuse=True) as scope:
        # With adversarial dropout
        logit_adD = modelWithAdD(x, y, n_channels=n_channels, is_train=is_train)
        logit_adD_loss = CrossEntropyWithLogits(logit_adD, y)

        # Total loss
        loss = logit_rand_loss + lmb * logit_adD_loss

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt = optimizer(learning_rate=learning_rate)
        gradients = opt.compute_gradients(loss, tf.trainable_variables())
        train_op = opt.apply_gradients(gradients)        

    return train_op, loss, logit_rand


def CreateTestModel(x, y, n_channels=3, is_train=True):  
    with tf.variable_scope(tf.get_variable_scope(), reuse=True) as scope:
        logit_rand = modelWithRandD(x, n_channels=n_channels, is_train=is_train)
        logit_rand_loss = CrossEntropyWithLogits(logit_rand, y)

        return logit_rand_loss, logit_rand


def Accuracy(logits, labels):
    y_pred = tf.argmax(logits, 1)
    y_true = tf.argmax(labels, 1)
    equality = tf.equal(y_pred, y_true)
    accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
    return accuracy


#### Visualization functions

In [0]:
'''
Uncomment to download graphs from google colab
'''
from google.colab import files

def visualize(trend, param):
    name = "Baseline" if param['BASELINE'] else "Adversarial"
    
    for key, val in trend.items():
        # X axis
        x = np.arange(len(val))
        # Acc or loss
        title = re.sub("_", " ", key).capitalize()
        plt.title(title + " " + name)
        plt.plot(x, val)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy' if 'acc' in key else 'Loss')
        filename = str(key + re.sub("{|}|:|'|,| ", "", str(param)) + ".png")
        print(filename)
        plt.savefig(filename)
        files.download(filename)
        plt.close()
        
        
def visualize_both(trend_base, trend_adv, param_base, param_adv):
    for key in trend_base.keys():
        # X axis
        x = np.arange(len(trend_base[key]))
        # Acc or loss
        title = re.sub("_", " ", key).capitalize()
        plt.title(title)
        plt.plot(x, trend_base[key], label='Base')
        plt.plot(x, trend_adv[key], label='Adversarial')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy' if 'acc' in key else 'Loss')
        plt.legend()
        filename = str(key + '_both' + ".png")
        print(filename)
        plt.savefig(filename)
        files.download(filename)
        plt.close()

#### Creates the graph with input/output nodes for training

In [0]:
def prepareTrainingModel(param):
    
    n_channels = param['CHANNELS']
    
    with tf.variable_scope('Baseline' if param['BASELINE'] else 'Adversarial'):
        # Graph
        x_train_ph = tf.placeholder(tf.float32)
        x_test_ph = tf.placeholder(tf.float32)
        y_train_ph = tf.placeholder(tf.float32)
        y_test_ph = tf.placeholder(tf.float32)

        train_op, train_loss, train_logit = CreateBaseModel(x_train_ph, y_train_ph, n_channels=n_channels, is_train=True) if param['BASELINE'] else CreateAdDModel(
            x_train_ph, y_train_ph, n_channels=n_channels, is_train=True)
        test_loss, test_logit = CreateTestModel(x_test_ph, y_test_ph, n_channels=n_channels, is_train=False)

        # Accuracy Train
        train_accuracy = Accuracy(train_logit, y_train_ph)

        # Accuracy Test
        test_accuracy = Accuracy(test_logit, y_test_ph)  
    
    nodes = {
        'x_train_ph': x_train_ph,
        'x_test_ph': x_test_ph,
        'y_train_ph': y_train_ph,
        'y_test_ph': y_test_ph,
        'train_op': train_op,
        'train_loss': train_loss,
        'train_logit': train_logit,
        'test_logit': test_logit,
        'test_loss': test_loss,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
    }
    
    return nodes
    

#### Can perform training incrementally i.e. you do not need to run all epochs at once

In [0]:
def doTraining(x_train, y_train, x_test, y_test, sess, nodes, param, trend, n_epochs=None):
    # Training setup

    batch_size = param['BATCH_SIZE']
    epochs = param['EPOCHS'] if n_epochs is None else n_epochs
    n_channels = x_train.shape[-1]

    STEPS = len(x_train) // batch_size if param['STEPS'] is None else param['STEPS']
    TEST_STEPS = len(x_test) // batch_size if param['STEPS'] is None else param['STEPS']
     
    # Training
    for epoch in range(epochs):
        # Train model
        acc_train, loss_train = 0, 0
        for i in range(STEPS):
            _, loss_, acc = sess.run([nodes['train_op'], nodes['train_loss'], nodes['train_accuracy']],
                                     feed_dict={nodes['x_train_ph']: x_train[batch_size * i: batch_size * (i + 1)],
                                                nodes['y_train_ph']: y_train[batch_size * i: batch_size * (i + 1)]})
            acc_train += acc
            loss_train += loss_

        trend['acc_train_trend'].append(acc_train / STEPS)
        trend['loss_train_trend'].append(loss_train / STEPS)

        # Test model
        acc_test, loss_test = 0, 0
        for i in range(TEST_STEPS):
            loss_t, acc_t = sess.run([nodes['test_loss'], nodes['test_accuracy']],
                                     feed_dict={nodes['x_test_ph']: x_test[batch_size * i: batch_size * (i + 1)],
                                                nodes['y_test_ph']: y_test[batch_size * i: batch_size * (i + 1)]})
            acc_test += acc_t
            loss_test += loss_t

        trend['acc_test_trend'].append(acc_test / TEST_STEPS)
        trend['loss_test_trend'].append(loss_test / TEST_STEPS)

        print('Epoch: {} || Train Loss: {}, Train Acc: {} || Test Loss: {}, Test Accuracy: {}'.format(epoch,
                                                                                                      loss_train / STEPS,
                                                                                                      acc_train / STEPS,
                                                                                                      loss_test / TEST_STEPS,
                                                                                                      acc_test / TEST_STEPS))
                  

#### Data I/O and preprocessing

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
x_train = np.expand_dims(x_train, axis = -1)
x_test = np.expand_dims(x_test, axis=-1)

In [0]:
# with tf.Session() as sess:
#     x_train = sess.run(tf.image.rgb_to_grayscale(x_train)) / 255
#     x_test = sess.run(tf.image.rgb_to_grayscale(x_test)) / 255
x_train, x_test = preprocess(x_train), preprocess(x_test)
    
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Adversarial Dropout training

In [0]:
param_adv = {
    'CHANNELS': 1,
    'BATCH_SIZE': 128,
    'EPOCHS': 100,
    'STEPS': None,
    'BASELINE': False
}

In [0]:
'''
Run this to delete default graph w/o having to restart notebook
'''
tf.reset_default_graph()

In [0]:
nodes_adv = prepareTrainingModel(param_adv)

In [0]:
trend_adv = {
    'acc_train_trend': [],
    'loss_train_trend': [],
    'acc_test_trend': [],
    'loss_test_trend': []
}

In [0]:
sess_adv = tf.Session()
sess_adv.run(tf.global_variables_initializer())

In [65]:
doTraining(x_train, y_train, x_test, y_test, sess_adv, nodes_adv, param_adv, trend_adv, n_epochs=100)

Epoch: 0 || Train Loss: 0.07901746204518913, Train Acc: 0.9760950854700855 || Test Loss: 0.1510687994484145, Test Accuracy: 0.9574318910256411
Epoch: 1 || Train Loss: 0.07408480551621169, Train Acc: 0.9776308760683761 || Test Loss: 0.18343957956546011, Test Accuracy: 0.946113782051282
Epoch: 2 || Train Loss: 0.0747570727347858, Train Acc: 0.9772636217948718 || Test Loss: 0.23171991319992605, Test Accuracy: 0.9341947115384616
Epoch: 3 || Train Loss: 0.06827763318328951, Train Acc: 0.9788828792735043 || Test Loss: 0.21476946090562984, Test Accuracy: 0.9375
Epoch: 4 || Train Loss: 0.06716755691025969, Train Acc: 0.9799679487179487 || Test Loss: 0.16741245377283448, Test Accuracy: 0.9542267628205128
Epoch: 5 || Train Loss: 0.0646234484967521, Train Acc: 0.9806523771367521 || Test Loss: 0.14784404564087686, Test Accuracy: 0.9605368589743589
Epoch: 6 || Train Loss: 0.0620915819382102, Train Acc: 0.9807024572649573 || Test Loss: 0.1487966660866872, Test Accuracy: 0.9596354166666666
Epoch: 7 |

Epoch: 36 || Train Loss: 0.03143161571429857, Train Acc: 0.9903679220085471 || Test Loss: 0.10832697785390803, Test Accuracy: 0.9714543269230769
Epoch: 37 || Train Loss: 0.03302410794917044, Train Acc: 0.9896167200854701 || Test Loss: 0.2012242215972107, Test Accuracy: 0.9516225961538461
Epoch: 38 || Train Loss: 0.03317039838290458, Train Acc: 0.9897669604700855 || Test Loss: 0.15457845292089936, Test Accuracy: 0.9636418269230769
Epoch: 39 || Train Loss: 0.03162877863667388, Train Acc: 0.9905348557692307 || Test Loss: 0.10286386542952058, Test Accuracy: 0.9747596153846154
Epoch: 40 || Train Loss: 0.031761798131507625, Train Acc: 0.9904013087606838 || Test Loss: 0.14700223217592934, Test Accuracy: 0.9672475961538461
Epoch: 41 || Train Loss: 0.03262013307278169, Train Acc: 0.9899338942307693 || Test Loss: 0.14667429778581628, Test Accuracy: 0.9671474358974359
Epoch: 42 || Train Loss: 0.031400835435150896, Train Acc: 0.9907518696581197 || Test Loss: 0.11720286465704423, Test Accuracy: 0.9

Epoch: 72 || Train Loss: 0.02526475533555547, Train Acc: 0.9922876602564102 || Test Loss: 0.11274072983696197, Test Accuracy: 0.9737580128205128
Epoch: 73 || Train Loss: 0.020495682663353457, Train Acc: 0.9933727297008547 || Test Loss: 0.12193603986219013, Test Accuracy: 0.9742588141025641
Epoch: 74 || Train Loss: 0.023027760130216693, Train Acc: 0.9933059561965812 || Test Loss: 0.14843361799452956, Test Accuracy: 0.9674479166666666
Epoch: 75 || Train Loss: 0.025030423978520965, Train Acc: 0.9921040331196581 || Test Loss: 0.15786012513742137, Test Accuracy: 0.965645032051282
Epoch: 76 || Train Loss: 0.02121223973536428, Train Acc: 0.9933059561965812 || Test Loss: 0.13457506136201586, Test Accuracy: 0.9739583333333334
Epoch: 77 || Train Loss: 0.021613190933650026, Train Acc: 0.9936565170940171 || Test Loss: 0.1606725969408543, Test Accuracy: 0.9670472756410257
Epoch: 78 || Train Loss: 0.022070360119674955, Train Acc: 0.9931724091880342 || Test Loss: 0.13675278700285184, Test Accuracy: 0

In [0]:
with open('adv_trend.json', 'w') as fp:
    json.dump(trend_adv, fp)
files.download('adv_trend.json')

In [0]:
visualize(trend_adv, param_adv)

acc_train_trendCHANNELS3BATCH_SIZE128EPOCHS50STEPSNoneBASELINEFalse.png
loss_train_trendCHANNELS3BATCH_SIZE128EPOCHS50STEPSNoneBASELINEFalse.png
acc_test_trendCHANNELS3BATCH_SIZE128EPOCHS50STEPSNoneBASELINEFalse.png
loss_test_trendCHANNELS3BATCH_SIZE128EPOCHS50STEPSNoneBASELINEFalse.png


## Baseline Training

In [0]:
param_base = {
    'CHANNELS': 1,
    'BATCH_SIZE': 128,
    'EPOCHS': 100,
    'STEPS': None,
    'BASELINE': True
}

In [0]:
nodes_base = prepareTrainingModel(param_base)

In [0]:
trend_base = {
    'acc_train_trend': [],
    'loss_train_trend': [],
    'acc_test_trend': [],
    'loss_test_trend': []
}

In [0]:
sess_base = tf.Session()
sess_base.run(tf.global_variables_initializer())

In [71]:
doTraining(x_train, y_train, x_test, y_test, sess_base, nodes_base, param_base, trend_base, n_epochs=100)

Epoch: 0 || Train Loss: 1.1412763578387408, Train Acc: 0.7677450587606838 || Test Loss: 6.350931124809461, Test Accuracy: 0.3345352564102564
Epoch: 1 || Train Loss: 0.33705859178772724, Train Acc: 0.9027276976495726 || Test Loss: 3.8171186324877615, Test Accuracy: 0.5291466346153846
Epoch: 2 || Train Loss: 0.2247361671179533, Train Acc: 0.9337606837606838 || Test Loss: 2.338742897296563, Test Accuracy: 0.6642628205128205
Epoch: 3 || Train Loss: 0.16878235779511622, Train Acc: 0.9485343215811965 || Test Loss: 1.8510233438931978, Test Accuracy: 0.7350761217948718
Epoch: 4 || Train Loss: 0.1460261662594146, Train Acc: 0.9558460202991453 || Test Loss: 0.6555017599692712, Test Accuracy: 0.8719951923076923
Epoch: 5 || Train Loss: 0.12228317553591397, Train Acc: 0.9630408653846154 || Test Loss: 0.7106628169616064, Test Accuracy: 0.8548677884615384
Epoch: 6 || Train Loss: 0.10874114975802855, Train Acc: 0.9668302617521367 || Test Loss: 0.2658417300703243, Test Accuracy: 0.9470152243589743
Epoc

Epoch: 36 || Train Loss: 0.035470650018412105, Train Acc: 0.9882979433760684 || Test Loss: 0.45416064732680983, Test Accuracy: 0.9603365384615384
Epoch: 37 || Train Loss: 0.03798578470878444, Train Acc: 0.9884314903846154 || Test Loss: 0.5300305628248615, Test Accuracy: 0.9598357371794872
Epoch: 38 || Train Loss: 0.03505497771281322, Train Acc: 0.9886985844017094 || Test Loss: 0.6701640195645805, Test Accuracy: 0.9559294871794872
Epoch: 39 || Train Loss: 0.036162798986171966, Train Acc: 0.9884982638888888 || Test Loss: 0.566128050263463, Test Accuracy: 0.9589342948717948
Epoch: 40 || Train Loss: 0.03524424920903825, Train Acc: 0.9892995459401709 || Test Loss: 1.0345396110071585, Test Accuracy: 0.9258814102564102
Epoch: 41 || Train Loss: 0.03487873560808446, Train Acc: 0.989082532051282 || Test Loss: 0.4871325163119163, Test Accuracy: 0.9623397435897436
Epoch: 42 || Train Loss: 0.031168693558216685, Train Acc: 0.9896167200854701 || Test Loss: 0.8645058341879541, Test Accuracy: 0.9393028

Epoch: 72 || Train Loss: 0.02561755399686721, Train Acc: 0.9919370993589743 || Test Loss: 1.097620500688739, Test Accuracy: 0.9574318910256411
Epoch: 73 || Train Loss: 0.025747363585418126, Train Acc: 0.9924045138888888 || Test Loss: 0.8118767844281859, Test Accuracy: 0.9693509615384616
Epoch: 74 || Train Loss: 0.022993762687541226, Train Acc: 0.9926048344017094 || Test Loss: 0.8412136378046565, Test Accuracy: 0.9705528846153846
Epoch: 75 || Train Loss: 0.024485324804550533, Train Acc: 0.9924379006410257 || Test Loss: 0.9082037869791949, Test Accuracy: 0.9678485576923077
Epoch: 76 || Train Loss: 0.022571357291874452, Train Acc: 0.9928886217948718 || Test Loss: 1.159375392204571, Test Accuracy: 0.9581330128205128
Epoch: 77 || Train Loss: 0.02481252411423786, Train Acc: 0.9921875 || Test Loss: 0.9447161417832924, Test Accuracy: 0.9683493589743589
Epoch: 78 || Train Loss: 0.022281886472953382, Train Acc: 0.9926716079059829 || Test Loss: 1.1639912793890774, Test Accuracy: 0.963341346153846

In [0]:
with open('base_trend.json', 'w') as fp:
    json.dump(trend_base, fp)
files.download('base_trend.json')

In [75]:
visualize(trend_base, param_base)

acc_train_trendCHANNELS1BATCH_SIZE128EPOCHS100STEPSNoneBASELINETrue.png
loss_train_trendCHANNELS1BATCH_SIZE128EPOCHS100STEPSNoneBASELINETrue.png
acc_test_trendCHANNELS1BATCH_SIZE128EPOCHS100STEPSNoneBASELINETrue.png
loss_test_trendCHANNELS1BATCH_SIZE128EPOCHS100STEPSNoneBASELINETrue.png


In [0]:
#@title Default title text
visualize_both(trend_base, trend_adv, param_base, param_adv)

## Trying more epochs for baseline

In [0]:
doTraining(x_train, y_train, x_test, y_test, sess_base, nodes_base, param_base, trend_base, n_epochs=15)

Epoch: 0 || Train Loss: 0.4114689419666926, Train Acc: 0.8565705128205128 || Test Loss: 2.62478596277726, Test Accuracy: 0.6821915064102564
Epoch: 1 || Train Loss: 0.4116022842434736, Train Acc: 0.8560296474358975 || Test Loss: 2.397842292602246, Test Accuracy: 0.6944110576923077
Epoch: 2 || Train Loss: 0.40900135078491306, Train Acc: 0.8569711538461539 || Test Loss: 2.573611721014365, Test Accuracy: 0.6738782051282052
Epoch: 3 || Train Loss: 0.3950696151990157, Train Acc: 0.8607371794871795 || Test Loss: 2.2602377350513754, Test Accuracy: 0.7008213141025641
Epoch: 4 || Train Loss: 0.3882401616909565, Train Acc: 0.864863782051282 || Test Loss: 2.585665913728567, Test Accuracy: 0.6643629807692307
Epoch: 5 || Train Loss: 0.37742966432601976, Train Acc: 0.8680889423076923 || Test Loss: 2.890182140545967, Test Accuracy: 0.6627604166666666
Epoch: 6 || Train Loss: 0.3734817562194971, Train Acc: 0.8680889423076923 || Test Loss: 3.031246014130421, Test Accuracy: 0.6494391025641025
Epoch: 7 || 